# Exemple d'extraction de graphe

## Setup

***Load modules*** 
This will require amtrack repo in the right path I use environment variables but feel free to do something better.
Normally all the required packages are indicated, if some are missing, please add them.
If you also want to create a "freeze" or a poetry thing to have an easier installation of the environment please do so.


In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from util import (
    get_dates_datetime,
    get_dirname,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
    fiji_path,
)
import ast
from plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    Edge,
    Node,
    plot_raw_plus,
)
from pipeline.paths.directory import (
    run_parallel_transfer,
    find_state,
    directory_scratch,
    directory_project,
    directory_archive,
)
import dropbox
from transfer.functions.transfer import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm
from time import time_ns
import subprocess
import matplotlib as mpl

***choose the path of interest***
I mounted sun on my linux computer at AMOLF. Please feel free to do so on yours as well. I can send you the command. 
Otherwise, if you have the folders locally, just indicate the path.
Finally you can also simply ssh to my computer and run things from there.

In [ ]:
# directory = r'/mnt/sun-temp/TEMP/PRINCE_felix/'
directory = r"/home/felix/Wks/amolf/"

update_plate_info(directory)
all_folders = get_current_folders(directory)

In [ ]:
len(all_folders)

***Choose the folder you're interested in***

In [ ]:
folders = all_folders.loc[all_folders["folder"] == "20220330_2357_Plate19"]

In [ ]:
folders

These are local adaptations of functions for running things on the supercomputer. So better to keep them in the notebook at this stage.

## Stiching function

In [ ]:
# path_code = os.getenv('HOME')+"/pycode/MscThesis/"
from util import path_code


def make_stitching_loop(directory, dirname, op_id):
    a_file = open(
        f"{path_code}amftrack/pipeline/scripts/stitching_loops/stitching_loop.ijm", "r"
    )

    list_of_lines = a_file.readlines()

    list_of_lines[4] = f"mainDirectory = \u0022{directory}\u0022 ;\n"
    list_of_lines[29] = f"\t if(startsWith(list[i],\u0022{dirname}\u0022)) \u007b\n"
    file_name = f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm'
    a_file = open(file_name, "w")

    a_file.writelines(list_of_lines)

    a_file.close()


def run_stitch(directory, folders):
    folder_list = list(folders["folder"])
    folder_list.sort()
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for folder in folder_list:
            op_id = time_ns()
            im = imageio.imread(f"{directory}/{folder}/Img/Img_r03_c05.tif")
            for x in range(1, 11):
                for y in range(1, 16):
                    strix = str(x) if x >= 10 else f"0{x}"
                    striy = str(y) if y >= 10 else f"0{y}"
                    path = f"{directory}/{folder}/Img/Img_r{strix}_c{striy}.tif"
                    # print(striy,path,os.path.getsize(path))
                    if not os.path.isfile(path):
                        f = open(path, "w")
                    if os.path.getsize(path) == 0:
                        imageio.imwrite(path, im * 0)
            make_stitching_loop(directory, folder, op_id)
            command = [
                fiji_path,
                "--mem=8000m",
                "--headless",
                "--ij2",
                "--console",
                "-macro",
                f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm',
            ]
            # modif 1
            print(" ".join(command))
            process = subprocess.run(command)
            pbar.update(1)


def run_parallel(code, args, folders):
    op_id = time_ns()
    folders.to_json(f'{os.getenv("TEMP")}/{op_id}.json')  # temporary file
    folder_list = list(folders["folder"])
    folder_list.sort()
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for index, folder in enumerate(folder_list):
            command = (
                [
                    "python",
                    f"{path_code}amftrack/pipeline/scripts/image_processing/{code}",
                ]
                + args_str
                + [f"{op_id}", f"{index}"]
            )
            print(" ".join(command))
            process = subprocess.run(command)
            pbar.update(1)

***Stitch the folders you're interested in***

should take about 20mins

In [ ]:
run_stitch(directory, folders)

***Compress the image and skeletonize***

should take 1h30-2h

In [ ]:
args = [directory]
run_parallel("compress_image.py", args, folders)

In [ ]:
time = "3:00:00"
low = 30
high = 80
extend = 30
args = [low, high, extend, directory]
run_parallel("extract_skel.py", args, folders)

## Visualisation

***Visualise the skeletonization***

In [ ]:
select = folders
folder_list = list(select["total_path"])
folder_list.sort()
skels = []
ims = []
kernel = np.ones((5, 5), np.uint8)
itera = 1
start = 0

finish = start + 2
for folder in folder_list[start:finish]:
    path_snap = folder
    skel_info = read_mat(path_snap + "/Analysis/skeleton_compressed.mat")
    skel = skel_info["skeleton"]
    skels.append(cv2.dilate(skel.astype(np.uint8), kernel, iterations=itera))
    im = read_mat(path_snap + "/Analysis/raw_image.mat")["raw"]
    ims.append(im)

In [ ]:
plt.close("all")
start = 0
finish = start + 1
for i in range(start, finish):
    plot_t_tp1([], [], None, None, skels[i], ims[i])

***These steps may be a bit long, but it also helps just keeping things consistent and use one single framework,*** you may bypass them by artificially renaming skeleton.mat into skeleton_realigned.mat which is necessary for the next step because you probably don't care about these two steps since you're doing only one image...

In [ ]:
time = "2:00"
thresh = 50
args = [thresh, directory]
run_parallel("mask_skel.py", args, folders)

In [ ]:
time = "3:00:00"
threshold = 0.1
args = [threshold, directory]
run_parallel("prune_skel.py", args, folders)

In [ ]:
plates = list(set(list(folders["Plate"].values)))
for plate in plates:
    time = "2:00:00"
    args = [directory]
    run_parallel("realign.py", args, folders)

Now you're finally getting to the point where you generate the graph object

In [ ]:
time = "2:00:00"
args = [directory]
run_parallel("extract_nx_graph.py", args, folders)

***and now you have the Experiment object to play with and query images related to position of edges etc...***


In [ ]:
run_info = folders
plate = int(list(run_info["folder"])[i].split("_")[-1][5:])
folder_list = list(run_info["folder"])
directory_name = folder_list[i]
exp = Experiment(plate, directory)
exp.load(run_info.loc[run_info["folder"] == directory_name], labeled=False)
path_snap = directory + directory_name
(G, pos) = exp.nx_graph[0], exp.positions[0]

In [ ]:
from random import choice

edge = choice(list(G.edges))

In [ ]:
edge_exp = Edge(Node(edge[0], exp), Node(edge[1], exp), exp)

***Visualiser l'image correspondant à l'edge*** 

***Visualiser tout le réseau avec l'edge idniqué dessus***

## Brouillon

In [ ]:
exp

In [ ]:
dir(exp)